In [25]:
import pandas as pd
import os
os.chdir("D:/demo/ML/demo_dominicks")

ana_sales = pd.read_parquet('data/raw/ana_sales_data.parquet')
ana_upc = pd.read_parquet('data/raw/ana_upc.parquet')
demographic_data = pd.read_parquet('data/raw/demographic_data.parquet')

## Построение датасета для первичной модели

1. В таблице продаж заменить None в колонке маркетинговых акций, удалить столбец qty (конкретно для этого вида товаров он всегда равен 1).
2. В таблице upc убрать столбцы case и nitem
3. В таблице demographic ничего не менять. Гипотезу о том, что исключение NaN улучшит модель оставим на одну из будущих итераций.
4. Все три таблицы смерджить в единый датасет

In [26]:
ana_sales.drop(columns='qty', inplace=True)
ana_sales.loc[ana_sales.sale.isna(),'sale'] = 'N'
ana_sales.sample(10)

,store,upc,week,move,price,sale
1565370,77,30045018615,138,4,4.99,N
6902107,129,32586610504,360,3,5.99,N
3170442,134,30573015042,295,0,0.00,N
4676976,12,31284310605,197,0,0.00,N
4876541,80,31284313818,280,0,0.00,N
2047618,112,30045046706,101,14,0.47,N
4003753,106,30573034040,183,0,0.00,N
6462076,110,31981007521,162,3,3.79,N
1503935,109,30045012490,393,0,0.00,N
2115889,70,30045046850,372,3,5.59,N


In [36]:
ana_upc.drop(columns=['case', 'nitem'], inplace=True)
ana_upc.sample(10)

,com_code,upc,descrip,size
22,953,3680012892,TC X/STR N/A PAIN RE,24 CT
602,953,34153300312,DAVOL MOIST HEATING,1 CT
468,953,31284319176,BAYER 8-HR ARTHRITIS,125 CT
618,953,35310012826,~SOMINEX TABLETS 16+,24 CT
622,953,35310012851,SOMINEX TABLETS 8658,72 CT
482,953,31284321614,PANADOL JR CAPLETS,30 CT
258,953,30087073304,TEMPRA A/F SYR CHLD,4 OZ
57,953,3828161073,DOM IBUPROFEN,50 CT
59,953,3828161077,DOM COATED ASPIRIN T,500 CT
373,953,31015804304,NYTOL W/DPH,32 CT


In [39]:
interim_df = pd.merge(ana_upc, ana_sales, on='upc', how='inner')
interim_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7241655 entries, 0 to 7241654
Data columns (total 9 columns):
 #   Column    Dtype  
---  ------    -----  
 0   com_code  int64  
 1   upc       int64  
 2   descrip   object 
 3   size      object 
 4   store     int64  
 5   week      int64  
 6   move      int64  
 7   price     float64
 8   sale      object 
dtypes: float64(1), int64(5), object(3)
memory usage: 497.2+ MB


In [40]:
baseline_df = pd.merge(interim_df, demographic_data, on='store', how='inner')
baseline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7173063 entries, 0 to 7173062
Data columns (total 21 columns):
 #   Column    Dtype  
---  ------    -----  
 0   com_code  int64  
 1   upc       int64  
 2   descrip   object 
 3   size      object 
 4   store     int64  
 5   week      int64  
 6   move      int64  
 7   price     float64
 8   sale      object 
 9   age60     float32
 10  age9      float32
 11  educ      float32
 12  ethnic    float32
 13  income    float32
 14  hhlarge   float32
 15  workwom   float32
 16  hval150   float32
 17  sstrdist  float32
 18  sstrvol   float32
 19  cpdist5   float32
 20  cpwvol5   float32
dtypes: float32(12), float64(1), int64(5), object(3)
memory usage: 820.9+ MB


In [44]:
baseline_df.sample(10)

,com_code,upc,descrip,size,store,week,move,price,sale,age60,...,educ,ethnic,income,hhlarge,workwom,hval150,sstrdist,sstrvol,cpdist5,cpwvol5
1358347,953,30009373501,~MOTRIN IB SINUS TAB,20 CT,137,302,2,4.59,N,0.209602,...,0.528362,0.113250,10.966490,0.092996,0.330293,0.860739,6.026484,0.705882,0.772530,0.333761
338720,953,3828161041,DOM IBUPROFEN,100 CT,47,122,0,0.00,N,0.125798,...,0.140599,0.120676,10.635326,0.145732,0.398027,0.335671,4.910911,2.028571,1.967745,0.794855
4266425,953,31284310111,BAYER ASPIRIN,24 CT,83,366,0,0.00,N,0.200835,...,0.145985,0.107628,10.456079,0.098154,0.330632,0.050757,1.346801,1.600000,3.901358,0.261861
2053803,953,30045046845,X/S TYLENOL GELCAP T,2-24 C,133,308,0,0.00,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2211379,953,30045047730,TYLENOL CHILD TABLET,30 CT,88,36,5,3.47,N,0.160414,...,0.151633,0.142928,10.549805,0.135168,0.401900,0.189573,4.981955,1.600000,2.087539,0.489796
5571082,953,31981000183,EXCEDRIN P M,50 CT,75,36,3,6.55,B,0.207699,...,0.219548,0.415999,9.867083,0.063965,0.315583,0.496000,7.192667,2.230769,1.375126,0.703182
5896483,953,31981000722,BUFFERIN CAPLET,60 CT,98,261,1,4.89,N,0.249201,...,0.051703,0.164964,10.573596,0.125409,0.299584,0.009843,6.230357,1.500000,3.133177,0.381569
1223228,953,30009348105,TRL-SZ MOTRINIB DIS,12 CT,50,42,0,0.00,N,0.153357,...,0.128764,0.070926,10.589308,0.111798,0.324213,0.240181,2.767046,0.727273,1.939329,0.470973
2420424,953,30045049650,TYLENOL TABLETS REG,50 CT,84,345,1,4.99,N,0.122100,...,0.188094,0.029636,10.765617,0.161592,0.346358,0.491441,0.966162,1.684211,1.582359,0.618634
1964356,953,30045045250,TYLENOL REG CAPLET,50 CT,110,179,0,0.00,N,0.114957,...,0.167553,0.152944,10.520860,0.161951,0.404314,0.232741,8.458997,0.954545,2.335458,0.321927


In [45]:
baseline_df.to_csv('data/baseline_dataset.csv')